<a href="https://colab.research.google.com/github/fiony/web_scrape/blob/master/PropertyListingReportGenerator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [248]:
SUB_LNS = 'Lower North Shore'
SUB_UNS = 'Upper North Shore'
SUB_MNS = 'Middle North Shore'
LNS_POSTCODE = 2069
UNS_POSTCODE = 2073

SUB_NE = 'North East'
SUB_SE = 'South East'
NE_POSTCODE = 2030

In [52]:
EMAIL_FIONY="fiony9@gmail.com"
EMAIL_DEBBIE="debbiemm@gmail.com"
EMAIL_BUS = [EMAIL_FIONY, EMAIL_DEBBIE]
EMAIL_DEV = [EMAIL_FIONY]

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [4]:
# import other libraries
!pip install import-ipynb
import import_ipynb

  Created wheel for import-ipynb: filename=import_ipynb-0.1.3-cp36-none-any.whl size=2976 sha256=27663dcc792b9786bdb3ca222eee43a4e77ef3a909f884a4a3515a38534dc9c7
  Stored in directory: /root/.cache/pip/wheels/b4/7b/e9/a3a6e496115dffdb4e3085d0ae39ffe8a814eacc44bbf494b5
Successfully built import-ipynb


In [5]:
%cd "/content/drive/My Drive/Colab Notebooks"
import LibDomainApi as domain
import HousingData as hd
import DomainApiClientInfo as ci

/content/drive/My Drive/Colab Notebooks
importing Jupyter notebook from LibDomainApi.ipynb
importing Jupyter notebook from HousingData.ipynb
importing Jupyter notebook from DomainApiClientInfo.ipynb


In [6]:
search_price = domain.PriceRange(1.8e6, 3e6)
estimate_ratio = 1.1
estimate_price = domain.PriceRange(search_price.min/estimate_ratio, search_price.max*estimate_ratio)
search_property = domain.PropertyRange(bed="3+", bath="1+", car="1+")

In [7]:
# this library is awesome: http://docs.python-requests.org/en/master/
import time,re, string, timeit, os, math, sys, logging, importlib, json, requests
from datetime import datetime
from collections import namedtuple
from IPython.display import display, HTML
import pandas as pd, numpy as np

In [ ]:
# reload in case update
#importlib.reload(domain)

importing Jupyter notebook from LibDomainApi.ipynb


<module 'LibDomainApi' from 'LibDomainApi.ipynb'>

In [8]:
domain.init_logging()

In [9]:
domain.setLogLevel(logging.INFO)

In [88]:
account = ci.accountA
client = domain.DomainClient(account.id, account.secret)

#### Load/Save records

In [11]:
DATA_PATH='/content/drive/My Drive/HouseListingResult'
def LoadExistingRecord(filename, path=DATA_PATH):
  df = pd.DataFrame()  
  file_path = os.path.join(path, filename)
  if os.path.exists(file_path):
    df = pd.read_csv(file_path, index_col=0) # first column as index
    UpdateFields(df)
  return df

In [12]:
COL_DOMAIN_PRICE = 'DomainPrice'
COL_MARKET_STATUS = 'status'
COL_BED_BATH_CAR = "BBC"
COL_GAS_POOL = "GP"
STATUS_ON_MARKET = 'ON'
STATUS_OFF_MARKET = 'OFF'
STATUS_NEW_MARKET = 'NEW'
EXTRA_FIELDS = {COL_DOMAIN_PRICE : None}
REPORT_COLUMNS_PREFIX = ['addr', COL_DOMAIN_PRICE, 'area']
REPORT_COLUMNS_POSFIX = ['price', 'sub']
HOUSE_COLUMNS_FULL = ['bed',	'car',	'bath',	'gas',	'pool']
REPORT_COLUMNS = REPORT_COLUMNS_PREFIX + HOUSE_COLUMNS_FULL + REPORT_COLUMNS_POSFIX
SHOW_COLUMNS = REPORT_COLUMNS + [COL_MARKET_STATUS]
COL_RENAME_MAP = {'addr':'Address','DomainPrice':'Price(Est)', 'area':'Area', 'price':'Guide', 'sub':'Suburb','bed':'Bed','car':'Car','bath':'Bath','gas':'Gas','pool':'Pool'}
COL_SHORTEN_NAME_MAP = {'gas':'G', 'Gas':'G', 'Pool':'P', 'pool':'P'}

In [80]:
def show(df_listing):
  display(HTML(domain.to_table(df_listing, show_column=SHOW_COLUMNS)))

def UpdateFields(df_listing, fields=EXTRA_FIELDS):
  for field, default_val in fields.items():
    if not field in df_listing.columns:
      df_listing[field] = default_val
  #return df_listing  

In [14]:
def reconcile(df_cur, df_last):
  df_all = df_cur.combine_first(df_last)
  df_all['status']=df_all.index.map(lambda x : STATUS_NEW_MARKET if x not in df_last.index else STATUS_OFF_MARKET if x not in df_cur.index else STATUS_ON_MARKET )
  return df_all

In [15]:
def SaveToCsv(df_listing, filename, update_latest=True, path=DATA_PATH):
  file_prefix, file_extension = os.path.splitext(filename)
  import shutil
  #filename = '%s.csv'%(file_prefix)
  date_str = datetime.now().strftime("%Y%m%d")
  backup_filename = '%s-%s%s'%(file_prefix, date_str, file_extension)
  df_listing.to_csv(filename)
  
  if update_latest:
    shutil.copyfile(filename, backup_filename)
    shutil.move(filename, os.path.join(path, filename))
    shutil.move(backup_filename, os.path.join(path, backup_filename))

#### Send Email

In [16]:
#https://docs.python.org/3.7/library/email.examples.html
def SendEmail(subject, to, html_body, txt_body=""):
  import smtplib  
  sender = "fiony.data@gmail.com"
  server = smtplib.SMTP('smtp.gmail.com', 587)
  server.starttls()
  server.login(sender, "Fiony9@data")

  # Import the email modules we'll need
  from email.message import EmailMessage
  from email.headerregistry import Address
  from email.utils import make_msgid  

  # Open the plain text file whose name is in textfile for reading.
  msg = EmailMessage()
  msg.set_content(txt_body)
  
  asparagus_cid = make_msgid()
  msg.add_alternative(
      html_body.format(asparagus_cid=asparagus_cid[1:-1]), 
      subtype='html')

  date_str = datetime.now().strftime("%d-%b-%Y")
  msg['Subject'] = "%s %s" % (subject, date_str)
  msg['From'] = Address("KeXueKanFang", "fiony.data", "gmail.com")
  msg['To'] = ','.join(to)

  server.send_message(msg)
  server.quit()

#### Send Report

In [241]:
def to_int(x):
  try:
    return int(x)
  except:
    return 'X'  # invalid

def shorten_bed_bath_car(df_listing):
  if 'bed' in df_listing.columns and 'bath' in df_listing.columns and 'car' in df_listing.columns:
    result = df_listing.copy()
    result[COL_BED_BATH_CAR] = result.apply(lambda x : "%s%s%s" % (to_int(x.bed), to_int(x.bath), to_int(x.car)), axis=1)
    return result.drop(columns=['bed','car','bath'])
  else:
    return df_listing

def shorten_gas_pool(df_listing):
  if 'gas' in df_listing.columns and 'pool' in df_listing.columns:
    result = df_listing.copy()
    result[COL_GAS_POOL] = result.apply(lambda x : "%s%s" % (x.gas, x.pool), axis=1)
    return result.drop(columns=['gas','pool'])
  else:
    return df_listing

def hyperlink_addr(df_listing, addr_col='addr', url_col='url'):
  hyperlink = lambda val : '<a target="_blank" href="{}">{}</a>'.format(val[0], val[1])
  df_listing[addr_col] = df_listing[[url_col, addr_col]].agg(hyperlink, axis=1)
  return df_listing

def column_rename(df, report_columns, use_short_colname):
  def _rename(name_map, name_list):
    return [name_map[x] if x in name_map else x for x in name_list]

  new_df = df.rename(columns=COL_RENAME_MAP)
  if use_short_colname:
    new_df = new_df.rename(columns=COL_SHORTEN_NAME_MAP)

  new_report = _rename(COL_RENAME_MAP, report_columns)
  if use_short_colname:
    new_report = _rename(COL_SHORTEN_NAME_MAP, new_report)

  return new_df, new_report

def to_html_table(df, show_column=[], use_short_colname=True, **kwargs):  
  _df = df.copy()

  _df = hyperlink_addr(_df)
  _df, show_column = column_rename(_df, show_column, use_short_colname)

  if show_column:
    _df = _df[show_column]

  _float_fmt = lambda x : "{:,.0f}".format(x)
  html = _df.to_html(escape=False, float_format=_float_fmt, **kwargs)
  return html      

In [18]:
Q_GAS = "gas=='Y'"
Q_NO_GAS = "gas=='N'"
Q_POOL = "pool=='Y'"
Q_NO_POOL = "pool=='N'"
Q_4_bed = "bed==4"
Q_5_bed = "bed>4"
Q_3_bed = "bed<4"
Q_OFF_MARKET = "%s=='%s'"%(COL_MARKET_STATUS, STATUS_OFF_MARKET)
Q_NOT_OFF_MARKET = "%s!='%s'"%(COL_MARKET_STATUS, STATUS_OFF_MARKET)
Q_NEW_MARKET = "%s=='%s'"%(COL_MARKET_STATUS, STATUS_NEW_MARKET)
Q_ON_MARKET = "%s=='%s'"%(COL_MARKET_STATUS, STATUS_ON_MARKET)

class ReportConfig(object):
  def __init__(self, title, queries=[], enrich_funcs=[], report_columns=REPORT_COLUMNS, short_colname=False):
    self.title = title
    self.queries = queries
    self.report_columns = report_columns
    self.enrich_funcs = enrich_funcs
    self.short_colname = short_colname

def ReportConfigByGP(title, queries):
  NOGP_REPORT_COLUMNS = REPORT_COLUMNS_PREFIX + [COL_BED_BATH_CAR] + REPORT_COLUMNS_POSFIX
  return ReportConfig(title=title, queries=queries, enrich_funcs=[shorten_bed_bath_car], report_columns=NOGP_REPORT_COLUMNS)

def ReportConfigByGas(title, queries):
  NOGAS_REPORT_COLUMNS = REPORT_COLUMNS_PREFIX + [COL_BED_BATH_CAR, 'pool'] + REPORT_COLUMNS_POSFIX
  return ReportConfig(title=title, queries=queries, enrich_funcs=[shorten_bed_bath_car], report_columns=NOGAS_REPORT_COLUMNS, short_colname=True)

COMPACT_FUNCS = [shorten_bed_bath_car, shorten_gas_pool]
COMPACT_REPORT_COLUMNS = REPORT_COLUMNS_PREFIX + [COL_BED_BATH_CAR, COL_GAS_POOL] + REPORT_COLUMNS_POSFIX
def ReportConfigCompact(title, queries):
  return ReportConfig(title=title, queries=queries, enrich_funcs=COMPACT_FUNCS, report_columns=COMPACT_REPORT_COLUMNS)

REPORT_SECTION = [
  ReportConfigByGP("4 Beds Gas Pool",   [Q_4_bed, Q_GAS, Q_POOL]),
  ReportConfigByGP("4 Beds Gas NoPool", [Q_4_bed, Q_GAS, Q_NO_POOL]),
  ReportConfigByGas("4 Beds NoGas",     [Q_4_bed, Q_NO_GAS]),
  ReportConfigByGas("3 Beds Gas",       [Q_3_bed, Q_GAS]),
  ReportConfigByGas("3 Beds NoGas",     [Q_3_bed, Q_NO_GAS]),
  ReportConfigByGas("5+ Beds Gas",      [Q_5_bed, Q_GAS]),
  ReportConfigByGas("5+ Beds NoGas",    [Q_5_bed, Q_NO_GAS]),
]

In [242]:
def SendReport(df_listing, subject, to, dry_run=False, sections=REPORT_SECTION):
  def _tag(t, v):
    return '<{tag}>{value}</{tag}>'.format(tag=t, value=v)

  x = {'html': '', 
       'txt' : "Oops, something went wrong, but we found %s houses on listing" % (len(df_listing))
       }

  def _update(df, title, enrich_funcs=COMPACT_FUNCS, report_columns=COMPACT_REPORT_COLUMNS, use_short_colname=False):
    for enrich in enrich_funcs:
      df = enrich(df)

    df = df.sort_values(by=[COL_DOMAIN_PRICE])    

    HTML_NEW_LINE = '<br><br>'
    TXT_NEW_LINE = '\r\n'

    x['html'] += _tag('h3', title)
    x['html'] += to_html_table(df, report_columns, use_short_colname, index=False)
    x['html'] += HTML_NEW_LINE

    x['txt'] += TXT_NEW_LINE + "%s:%s"%(title, len(df))

  df_off_market = df_listing.query(Q_OFF_MARKET)
  df_new_market = df_listing.query(Q_NEW_MARKET)
  df_market = df_listing.query(Q_ON_MARKET)

  MAX_NEW_LISTING = 5
  if len(df_new_market) > MAX_NEW_LISTING:
    df_market = pd.concat([df_new_market, df_market])
    df_new_market = []

  if len(df_new_market)>0:
    _update(df_new_market, 'New Listing')
  if len(df_off_market)>0:
    _update(df_off_market, 'Off Market')

  for s in sections:
    df = df_market
    for q in s.queries:
      df = df.query(q)    
    if len(df)>0:
      _update(df, s.title, s.enrich_funcs, s.report_columns, s.short_colname)  

  if not dry_run: 
    SendEmail(subject, to, x['html'], x['txt'])
    print(to)
  else:
    print(x['txt'])

#### Update new listing and price

In [21]:
def get_house_listing(subs, price_range=None, property_range=None):
  result = []

  for sub in subs:
    print("Processing %s %s"%(sub.name, sub.postcode))
    sub_by_price = client.listing(sub, price_range=price_range, property_range=property_range, expand_search=False)    
    if sub_by_price: 
      result += sub_by_price
    domain.log(logging.INFO, 'Processing %s: return %s results. Total result %s',sub.name, len(sub_by_price), len(result))      

  df = domain.to_dataframe(result)
  UpdateFields(df)
  return df

In [22]:
def get_listing_by_postcodes(postcodes, price_range=None, property_range=None, state='NSW'):
  subs = [domain.Suburb("", x, state) for x in postcodes]
  return get_house_listing(subs, price_range, property_range)

def get_listing_by_suburb(suburbs, price_range=None, property_range=None, state='NSW'):
  subs = suburbs.apply(lambda row: domain.Suburb(row.suburb, row.postcode, state), axis=1).tolist()
  return get_house_listing(subs, price_range, property_range)

In [23]:
def get_domain_price(prop_id, prop_info, estimate_range, estimate_error=5000):
  def is_in_price_range(min_price, max_price):
    criteria = {
      "listingType":"Sale",
      "minPrice":int(min_price),
      "maxPrice":int(max_price),
      "propertyTypes":['House'],
      "minBedrooms":int(prop_info['bed']),
      "maxBedrooms":int(prop_info['bed']),
      "minBathrooms":int(prop_info['bath']),
      "maxBathrooms":int(prop_info['bath']),
      "minCarspaces":int(prop_info['car']),
      "maxCarspaces":int(prop_info['car']),
      "locations":[
        {
          "state":'NSW',
          "region":"",
          "area":"",
          "suburb":prop_info['sub'],
          "postCode":int(prop_info['postcode']) if 'postcode' in prop_info else "",
          "includeSurroundingSuburbs":False
        }
      ]
    }
    result = client.listing_by_criteria(criteria)
    return any(prop_id==x[0] for x in result)

  min_price = estimate_range.min if estimate_range.min else 0
  max_price = estimate_range.max if estimate_range.max else 0
  mid_price = None
  domain.log(logging.INFO, '[START][%s] min=%s max=%s', prop_id, min_price, max_price)

  while(min_price and max_price and min_price + estimate_error < max_price):
    mid_price = min_price + int((max_price - min_price)/2)
    domain.log(logging.DEBUG, '[SEARCH] min=%s max=%s mid=%s', min_price, max_price, mid_price)
    if is_in_price_range(min_price, mid_price):
      max_price = mid_price
    elif is_in_price_range(mid_price, max_price): 
      min_price = mid_price
    else:
      mid_price = None
      domain.log(logging.WARNING, '[FAILED] min=%s max=%s mid=%s', min_price, max_price, mid_price)
      break

  return mid_price, min_price, max_price

In [281]:
def is_invalid_price(price):
  return price is None or math.isnan(price) or price <= 0

def round_price(df_listing, tens=4):
  df_listing[COL_DOMAIN_PRICE] = df_listing[COL_DOMAIN_PRICE].apply(lambda x : round(x, -tens) if x else x)

def verify_price(df_listing, price_result, to_update=False):
  mismatch = 0
  for pid, price in price_result.items():
    df_price = df_listing.loc[pid, COL_DOMAIN_PRICE]
    if df_price != price:
      mismatch += 1
      print('Price Not Match: pid=%s result=%s df_price=%s'%(pid, price, df_price))
      if to_update:
        df_listing.at[pid, COL_DOMAIN_PRICE] = price
  print('PASS' if mismatch == 0 else "%s mismatch"%(mismatch))
  return mismatch  

def update_domain_price(df_listing, estimate_price_range, pid_list=[], exclude_list=[], force_update=False, estimate_error=1e4, run_limit=None):
  result = {}
  MAX_FAIL = 3
  MAX_TO_SAVE = 5
  processed = 0
  failed =  0
  to_save = 0
  for pid in df_listing.index:
    try:
      if pid_list and pid not in pid_list:
        continue

      if exclude_list and pid in exclude_list:
        continue

      if not force_update and not is_invalid_price(df_listing.loc[pid][COL_DOMAIN_PRICE]):
        continue
      
      processed += 1

      prop_info = df_listing.loc[pid].to_dict()
      mid, min, max = get_domain_price(pid, prop_info, estimate_price_range, estimate_error)
      df_listing.at[pid,COL_DOMAIN_PRICE] = mid
      result[pid] = mid

      to_save += 1

      domain.log(logging.INFO, "[PRICE][%s] %s (%s - %s)", pid, mid, min, max)
    except:
      failed += 1
      domain.log(logging.WARNING, "Failed to get price for [%s], total failed=%s", pid, failed)

    if to_save >= MAX_TO_SAVE:
      mismatch = verify_price(df_listing, result)
      if mismatch == 0:
        print("Save Price after %s price found" % to_save)
        df_to_save = df_listing.query(Q_NOT_OFF_MARKET)
        SaveToCsv(df_to_save, record_file, update_latest=False)
        to_save = 0
      else:
        print("TO CHECK: found %s price mismatch" % mismatch)
        break
      
    if run_limit and processed >= run_limit:
      print("Done for processing %s property" % run_limit)
      break

    if failed >= MAX_FAIL:
      print("Stop after failed %s times" % failed)
      break

  return result
  #df_listing[COL_DOMAIN_PRICE] = df_listing.index.map(lambda x : domain.PropertyReport(client, x).check_price(estimate_price_range,estimate_error)[0])

In [25]:
def get_pid_list_without_price(df_listing):
  result = []
  df_market = df_listing.query(Q_NOT_OFF_MARKET)
  for section in REPORT_SECTION:
    df = df_market
    for q in section.queries:
      df = df.query(q)
    idx_invalid_price = df[COL_DOMAIN_PRICE].apply(is_invalid_price)
    result += df[idx_invalid_price].index.tolist()
  return result

### Setup

In [249]:
def get_record_filename(subject, file_ext='csv'):  
  prefix = '_'.join(subject.lower().split(' '))
  return  "%s.%s" % (prefix, file_ext)

def setup_suburb():
  result = {}

  # north shore  
  suburbs = hd.LoadSydneySuburbs('north shore')
  result[SUB_LNS] = suburbs[suburbs.postcode < LNS_POSTCODE]
  result[SUB_MNS] = suburbs[(suburbs.postcode >= LNS_POSTCODE) & (suburbs.postcode <= UNS_POSTCODE)]
  result[SUB_UNS] = suburbs[suburbs.postcode > UNS_POSTCODE]

  # EAST
  suburbs = hd.LoadSydneySuburbs('east')
  result[SUB_NE] = suburbs[suburbs.postcode < NE_POSTCODE]
  result[SUB_SE] = suburbs[suburbs.postcode >= NE_POSTCODE]

  return result

### Listing

In [250]:
region = SUB_NE

In [251]:
all_suburbs = setup_suburb()

In [252]:
suburb = all_suburbs[region]
record_file = get_record_filename(region)
print(region, record_file)

North East north_east.csv


In [ ]:
#df_last = LoadExistingRecord('lower_north_shore-20200705.csv')
#df_cur = LoadExistingRecord(record_file)

In [253]:
df_last = LoadExistingRecord(record_file)
print('Load Existing: %s' % len(df_last))

Load Existing: 0


In [254]:
# prepare only postcode
postcodes= [int(x) for x in suburb.postcode.unique()]
print(postcodes)

[2020, 2021, 2022, 2023, 2024, 2025, 2026, 2027, 2028, 2029]


In [255]:
df_cur = get_listing_by_postcodes(postcodes, search_price)

Processing  2020


13:36:05,878 DomainApi INFO Processing : return 3 results. Total result 3


Processing  2021


13:36:08,461 DomainApi INFO Processing : return 14 results. Total result 17


Processing  2022


...

13:36:23,127 DomainApi INFO Processing : return 0 results. Total result 38


Processing  2028


13:36:25,630 DomainApi INFO Processing : return 0 results. Total result 38


Processing  2029


13:36:27,937 DomainApi INFO Processing : return 3 results. Total result 41


In [256]:
df_all = reconcile(df_cur,df_last)
print('Combined : %s'%len(df_all))

Combined : 41


In [258]:
#df_all[df_all[COL_DOMAIN_PRICE].isna()]


#### Update Price

In [261]:
pid_to_update = get_pid_list_without_price(df_all)
print("%s out %s needs price update" % (len(pid_to_update), len(df_all)))

41 out 41 needs price update


In [ ]:
#run_pid = df_all[df_all.DomainPrice==2400000].index.tolist()
#len(run_pid)

In [262]:
select_pids = pid_to_update

In [275]:
exclude_pids = [2016380074,2016363574,2016315628,2016107573,2016215503,2016282966]

In [276]:
price_range_to_use = search_price 
price_result = update_domain_price(df_all, price_range_to_use, select_pids, exclude_list=exclude_pids, run_limit=10)

13:47:27,141 DomainApi INFO [START][2016268490] min=1800000.0 max=3000000.0
13:47:27,142 DomainApi WARNING Failed to get price for [2016268490], total failed=1
13:47:27,145 DomainApi INFO [START][2016317206] min=1800000.0 max=3000000.0
13:47:27,147 DomainApi WARNING Failed to get price for [2016317206], total failed=2
13:47:27,149 DomainApi INFO [START][2016323276] min=1800000.0 max=3000000.0
13:47:27,150 DomainApi WARNING Failed to get price for [2016323276], total failed=3


Stop after failed 3 times


In [277]:
verify_price(df_all, price_result, to_update=False)

PASS


0

#### Save File and Send Report

In [239]:
df_all[df_all[COL_DOMAIN_PRICE].isna()]

,DomainPrice,addr,area,auction,bath,bed,car,gas,inspect,latitude,longitude,pool,postcode,price,status,sub,url
pid,,,,,,,,,,,,,,,,,


In [278]:
df_to_save = df_all.query(Q_NOT_OFF_MARKET)

In [279]:
SaveToCsv(df_to_save, record_file, update_latest=True)

In [282]:
round_price(df_all)

In [283]:
SendReport(df_all, subject=region, to=EMAIL_DEV, dry_run=False)

['fiony9@gmail.com']


In [268]:
show(df_all[0:5])

,addr,DomainPrice,area,bed,car,bath,gas,pool,price,sub,status
pid,,,,,,,,,,,
2016272166,19 Sparks Street,"1,903,125",591,6,3,3,Y,Y,"For Sale $1,900,000 - $2,000,000",MASCOT,NEW
2016295699,14 Francis Street,"1,846,875",nan,5,2,3,Y,N,"$1,900,000 - $2,000,000",MASCOT,NEW
2015399273,2 and 4 Walker Ave,"2,090,625",562,6,1,3,N,N,2 Homes 1 Title,MASCOT,NEW
2016380074,39 Harris Street,None,nan,1,nan,2,N,N,Auction - 15th August,PADDINGTON,NEW
2016282976,10 Moore Park Road,"2,990,625",220,3,3,2,Y,N,Contact Agent,PADDINGTON,NEW


### Check Single Property

In [ ]:
#prop = domain.PropertyReport(client, 2016319850)

In [ ]:
#prop.domain_price(price_range=domain.PriceRange(2e6, 5e6))

#### DEBUG

In [ ]:
"""
i=0
while(i<1000):
  print(i)
  i+=1
  time.sleep(30)
"""

'\ni=0\nwhile(i<1000):\n  print(i)\n  i+=1\n  time.sleep(30)\n'